In [ ]:
# import sys
# import gymnasium as gym
# sys.modules["gym"] = gym
import gym
gym.__version__

In [ ]:
import os
import numpy as np
np.random.seed(0)
import torch
print(torch.cuda.is_available())

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback

In [ ]:
env_id = "MountainCarContinuous-v0"
NUM_CPU = 32  # Number of processes to use

In [ ]:
# Parallel environments
train_env_f = make_vec_env(env_id, n_envs=NUM_CPU)
train_env_f.observation_space

In [ ]:
log_prefix = './test2kk/'

In [ ]:
# for i in range(10):
for i in np.random.randint(2**16, size=10):
    model = PPO("MlpPolicy", 
            train_env_f,
            verbose=1, 
            seed=int(i),
            batch_size=512,
            learning_rate=7.77e-05,
            n_epochs=10,
            n_steps=8*NUM_CPU,
            gae_lambda=0.9,
            gamma=0.9999,
            max_grad_norm=5,
            use_sde=True,
            tensorboard_log= log_prefix + "logs/ppo_MountainCar_tensorboard_expert/")
    
    eval_callback = EvalCallback(train_env_f, 
                                 best_model_save_path= log_prefix + f"best_models/expert{i}",
                                 log_path= log_prefix + f"results/expert{i}",
                                 eval_freq=2**14/NUM_CPU,
                                 deterministic=True, render=False)

    model.learn(total_timesteps=2_000_000, callback=eval_callback, progress_bar=True)
    model.save(log_prefix + f"/models/expert{i}")

In [ ]:
train_env = make_vec_env(env_id, 
                         env_kwargs=dict(
                            full_obs=False
                         ),
                         n_envs=NUM_CPU)
train_env.observation_space

In [ ]:
# for i in range(10):
for i in np.random.randint(2**16, size=10):
    model = PPO("MlpPolicy", 
            train_env,
            verbose=1, 
            seed=int(i),
            batch_size=512,
            learning_rate=7.77e-05,
            n_epochs=10,
            n_steps=8*NUM_CPU,
            gae_lambda=0.9,
            gamma=0.9999,
            max_grad_norm=5,
            use_sde=True,
            tensorboard_log= log_prefix + "logs/ppo_MountainCar_tensorboard_noob/")
    
    eval_callback = EvalCallback(train_env, 
                                 best_model_save_path=log_prefix + f"best_models/noob{i}",
                                 log_path=log_prefix + f"results/noob{i}",
                                 eval_freq=2**14/NUM_CPU,
                                 deterministic=True, render=False)

    model.learn(total_timesteps=2_000_000, callback=eval_callback, progress_bar=True)
    model.save(log_prefix + f"models/noob{i}")